<a href="https://colab.research.google.com/github/Ismail-Armutcu/Algorithms-for-Interactive-Sytems/blob/ders_kod/MMI513_week5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 5: Tournaments**

This notebook contains implementations of several tournament types, including

- Rank Adjustment Tournaments (including Ladder, Hill-climbing, Pyramid) with initial rank adjustment
- Single Elimination Tournaments (also including standard seeding, ordered standard seeding, and equitable seeding algorithms)
- Scoring Tournaments



### Preamble

- We will only need some functions from `numpy` and its submodules in this notebook.

In [ ]:
import numpy as np

# Rank Adjustment Tournaments

In *rank adjustment tournaments*, players are initially ranked based on their previous performance. Then, they play a number of matches against opponents with similar rankings, and their rankings are adjusted based on the results. This method is useful for situations where the rankings of the players are already established, but need to be updated based on recent performances. In this section, we will implement a Rank Adjustment Tournament in Python, where players are initially seeded based on their previous rankings, and then play matches against each other. The results of each match will be used to adjust the rankings of the players, and the final ranking will be determined based on the adjusted scores.


## Initial rank adjustment and Ladder tournament

- Ladder tournaments are a popular format of competition in which players are ranked based on a series of head-to-head matches. The concept of a ladder tournament is simple: players are initially ranked in order, and they move up or down the ladder based on their performance in matches against other players.

- In this section, we will implement a ladder tournament using Python. We will also discuss the initial ranking of players, which is an important aspect of ladder tournaments.

- The code included in this subsection defines several functions that we will use to simulate a ladder tournament. These functions allow us to create unranked players, enumerate players in random order, rank players, set ranks, and simulate matches between players. We will use these functions to implement a ladder tournament and adjust player ranks based on match outcomes.


- `unrankedplayers(n)`:
  - Creates a list of n unranked players with the format `P<index>`.
  - Returns a dictionary with two keys:
    - "P": a list of player names.
    - "rnks": a list of None values indicating that the players have not been ranked yet.
- `enumeration(P)`:
  - Randomly shuffles the order of the players in P.
  - Returns a dictionary with two keys:
    - "P": a shuffled list of player names.
    - "rnks": a shuffled list of None values indicating that the players have not been ranked yet.
- `rankeds(P, r)`:
  - Filters the players in P who have a rank of r.
  - Returns a dictionary with two keys:
    - "P": a list of player names with rank r.
    - "rnks": a list of rank r.
- `setrank(P, index, r)`:
  - Copies the dictionary P.
  - Sets the rank of the player at index index to r.
  - Returns the modified copy of P.
- `match(p, q, thr, tieflag=True)`:
  - Simulates a match between player p and player q with a threshold of thr (a number between 0 and 1).
  - If tieflag is True, ties are allowed, otherwise, they are not.
  - Returns the name of the winning player or `None` if there is a tie.
- `ladder_match(P, p, q)`:
  - Simulates a match between player p and player q using the match function.
  - If q wins, swaps the ranks of p and q in P.
  - Returns the modified dictionary P.
- `initial_rank_adjustment(P, S)`:
  - Copies the dictionary P and shuffles the order of the players using enumeration.
  - Assigns the initial champion (rank 0) to the first `S[0]` players.
  - For each subsequent rank r, simulates matches between the top players from rank r-1 to determine the players who will be assigned rank r.
  - Returns the modified dictionary P.

In [ ]:
def unrankedplayers(n):
    P = []
    rnks = []
    for ind in range(n):
        P.append('P' + str(ind))
        rnks.append(None)
    return {'P': P, 'rnks': rnks}

def enumeration(P):
    lst = np.arange(len(P['P']))
    order = np.random.permutation(lst)
    M = lst[order]
    return {'P':[P['P'][ind] for ind in order], 'rnks':[P['rnks'][ind] for ind in order]}

def rankeds(P, r):
    W = {'P':[], 'rnks':[]}
    for ind in range(len(P['rnks'])):
        if P['rnks'][ind] == r:
            W['P'].append(P['P'][ind])
            W['rnks'].append(P['rnks'][ind])
    return W

def setrank(P, index, r):
    R = P.copy()
    R['rnks'][index] = r
    return R

def match(p, q, thr, tieflag=True):
    toss = np.random.rand(1)
    upC = 1
    downC = 1
    if tieflag: # If Tie is allowed
      upC = 1.1
      downC = 0.9
    if thr * downC < toss < thr * 1.1: # If the random number is around thr, result is a tie
        result = None # Tie
    elif toss > thr * upC:
        result = p
    else:
        result = q
    return result

def ladder_match(P, p, q):
    m = match(p, q, 0.5)
    if m == None or m == p:
        return P
    else:
        indp = P['P'].index(p)
        indq = P['P'].index(q)
        rankp = P['rnks'][indp]
        rankq = P['rnks'][indq]
        if rankq == None:
            rankq = rankp
        P['rnks'][indp] = rankq
        P['rnks'][indq] = rankp
        return P

def initial_rank_adjustment(P, S):
    assert len(P['P']) == sum(S)
    m = len(S)
    R = P.copy()
    R = enumeration(R)
    for ind in range(S[0]):
        R['rnks'][ind] = 0 # The initial champion(s)
    c =  S[0]

    for rank in range(1, m):
        W = rankeds(R, rank - 1) # The runners-up
        Mp = enumeration(W)
        for ind in range(S[rank]):
            setrank(R, c + ind, rank)
            jnd = ind % len(Mp['P'])
            if Mp['rnks'][jnd] != rank:
                R = ladder_match(R, Mp['P'][jnd], M['P'][c + ind])
        c += S[rank]
    return R

## Testing code ✅

In [ ]:
P = unrankedplayers(11)
print(P)
S = [1, 2, 3, 5] # Ranking structure: One champion, 2 second places, etc.
M = enumeration(P)
R = initial_rank_adjustment(P, S)
print(R)

{'P': ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10'], 'rnks': [None, None, None, None, None, None, None, None, None, None, None]}
{'P': ['P3', 'P6', 'P0', 'P1', 'P7', 'P4', 'P10', 'P2', 'P8', 'P9', 'P5'], 'rnks': [0, 1, 3, 1, 2, 3, 3, 2, 3, 2, 3]}


## Hill-climbing tournaments

Hill Climbing Tournaments are a type of tournament where players are ranked based on their performance against other players of similar rank. In this tournament, players start at the bottom rank and climb up the ladder by winning matches against higher-ranked opponents. The players who win the most matches and climb the highest on the ladder are considered the winners.

`hillclimbing(P)` function takes as input a dictionary `P` containing the players and their initial ranks, and returns a modified dictionary `R` containing the players and their final ranks, as well as a list of matches containing tuples of players who played against each other. The function works as follows:

- First, the function initializes a list S containing the number of places for each rank, which is set to one for each rank since each rank has only one place in the Hill Climbing Tournament.
- The function then uses the `initial_rank_adjustment` function to adjust the initial ranks of the players based on the `S` list.
- The function then iterates over each rank, from the bottom to the top, and finds the opponent for the current player at that rank.
- It then adds the match between the player and their opponent to the matches list.
- The function then uses the `ladder_match` function to simulate the winner of the match and update the ladder accordingly.
- Finally, the function returns the modified `R` dictionary and the list of matches that happened in the tournament.

In [ ]:
def hillclimbing(P):
  n = len(P['P'])
  matches = []
  S = list(np.ones(n, dtype=int)) # Each rank has only one place
  R = initial_rank_adjustment(P,S)
  c = rankeds(R, n - 1)
  for rp in range(n-1):
    r = n - 2 - rp # For each rank from the bottom to the top
    opp = rankeds(R, r)
    matches.append((opp['P'][0], c['P'][0]))
    R = ladder_match(R, opp['P'][0], c['P'][0]) # Because each rank has only one place
    c = rankeds(R, r)
  return R, matches

## Testing code ✅

In [ ]:
R, matches = hillclimbing(P)
print(R)
print(matches)

{'P': ['P3', 'P0', 'P1', 'P4', 'P10', 'P5', 'P9', 'P2', 'P8', 'P6', 'P7'], 'rnks': [2, 6, 0, 4, 3, 1, 7, 8, 5, 10, 9]}
[('P6', 'P7'), ('P8', 'P7'), ('P2', 'P8'), ('P9', 'P8'), ('P0', 'P8'), ('P10', 'P8'), ('P4', 'P10'), ('P3', 'P10'), ('P5', 'P3'), ('P1', 'P5')]


## Pyramid Tournament

Pyramid Tournaments are a type of ranking tournament that utilizes a pyramid structure to determine the overall winner. In this format, players are initially divided into groups of equal sizes, with the best player at the top of the pyramid and the rest of the players forming lower levels of the pyramid.

Each level of the pyramid is a mini tournament, and the winner of each level proceeds to the next level of the pyramid. The last level of the pyramid determines the overall winner of the tournament. The advantage of this structure is that it allows players to compete against others of similar skill levels, making the tournament more fair and exciting.

The Python code provided below is for the pyramid match function. This function is used to determine the winner of a match between two players in a Pyramid Tournament. The function takes in three arguments: the tournament data P, and the indices of the players p and q. The function checks whether the two players are eligible to play a match based on their ranks, and if so, conducts a match and updates the tournament data. If not, the function returns the tournament data unchanged.

**NOTE:**
Your professor is not proud about how he implemented `pyramid_match()`. This was due to combination of a lack of oversight. A much better approach might have been defining a `Player` class.

In [ ]:
def pyramid_match(P, p, q):

  pi = P['P'].index('P'+str(p))
  qi = P['P'].index('P'+str(q))

  rank_p = P['rnks'][pi]
  peerWinner_p = P['peerWinner'][pi]
  rank_q = P['rnks'][qi]
  peerWinner_q = P['peerWinner'][qi]

  print((rank_p == rank_q) and (not peerWinner_q))
  print((rank_p == rank_q - 1) and peerWinner_q)

  if not ((rank_p == rank_q) and (not peerWinner_q)) | ((rank_p == rank_q - 1) and peerWinner_q):
    print('No match was played! Ranks do not match the conditions for pyramid match!')
    return P

  R = P.copy() # We copy the player dictionary first
  R['P'].pop(pi)
  R['P'].pop(qi)
  R['rnks'].pop(pi)
  R['rnks'].pop(qi)
  R['peerWinner'].pop(pi)
  R['peerWinner'].pop(qi)

  print(R)

  # We do not have the two players in our set of players now.
  m = match(p, q, 0.5) # Let them compete

  if (rank_p == rank_q): # Peer match
    if (((m == p) and (not peerWinner_p)) or (((m == q) or m == None) and peerWinner_p)):
      pp = p
      rank_pp = rank_p
      peerWinner_pp = (m == p)
    else:
      pp = p
      rank_pp = rank_p
      peerWinner_pp = peerWinner_p
    if (m == q):
      qp = q
      rank_qp = rank_q
      peerWinner_qp = True
    else:
      qp = q
      rank_qp = rank_q
      peerWinner_qp = peerWinner_q

    R['P'].append('P'+str(pp))
    R['P'].append('P'+str(qp))
    R['rnks'].append(rank_pp)
    R['rnks'].append(rank_qp)
    R['peerWinner'].append(peerWinner_pp)
    R['peerWinner'].append(peerWinner_qp)
    return R
  else: # Rank challenge match
    qp = q
    rank_qp = rank_q
    peerWinner_qp = False
    if (m == p) or m == None:
      R['P'].append('P'+str(p))
      R['P'].append('P'+str(qp))
      R['rnks'].append(rank_p)
      R['rnks'].append(rank_qp)
      R['peerWinner'].append(peerWinner_p)
      R['peerWinner'].append(peerWinner_qp)
      return R
    else:
      pp = p
      peerWinner_pp = False
      rank_pp = rank_q
      rank_qp = rank_p
      R['P'].append('P'+str(pp))
      R['P'].append('P'+str(qp))
      R['rnks'].append(rank_pp)
      R['rnks'].append(rank_qp)
      R['peerWinner'].append(peerWinner_pp)
      R['peerWinner'].append(peerWinner_qp)
      return R

## Testing code ✅

In [ ]:
n = 11
P = unrankedplayers(n)
S = [1, 2, 3, 5]
R = initial_rank_adjustment(P, S)
# Appending the peerWinner field to R makes it into pyramid structure
R['peerWinner'] = [True] * n
print(R)

{'P': ['P8', 'P5', 'P3', 'P7', 'P1', 'P10', 'P9', 'P4', 'P0', 'P2', 'P6'], 'rnks': [0, 1, 3, 2, 1, 3, 2, 2, 3, 3, 3], 'peerWinner': [True, True, True, True, True, True, True, True, True, True, True]}


In [ ]:
R = pyramid_match(R, 3, 4)
print(R)

False
False
No match was played! Ranks do not match the conditions for pyramid match!
{'P': ['P8', 'P5', 'P3', 'P7', 'P1', 'P10', 'P9', 'P4', 'P0', 'P2', 'P6'], 'rnks': [0, 1, 3, 2, 1, 3, 2, 2, 3, 3, 3], 'peerWinner': [True, True, True, True, True, True, True, True, True, True, True]}


##Elimination Tournaments

Another type of tournament commonly used in competitions is the elimination tournament. In an elimination tournament, players are eliminated after losing one or more matches, until only one player remains as the winner.

### Random Selection Tournament

`randomselectiontournament(P, W)` selects a player randomly from the set of players `P` with probabilities proportional to the weight vector `W`.
- The weight vector `W` is updated in the `randompairingtournament` function.
- The function `randompairingtournament(P)` implements the random pairing elimination tournament.
  - It first enumerates the players in `P`, generates random pairings for each round, and updates the weight vector `W` based on the results of the matches. Finally, it selects a player randomly from the remaining players using the `randomselectiontournament` function.

In [ ]:
def randomselectiontournament(P, W):
  p = np.array(W)/np.sum(W)
  R = P.copy()
  k = np.random.choice(len(P['P']), 1, list(p))
  kind = k[0]
  indk = R['P'].index('P'+ str(kind))
  R['rnks'] = [1] * len(R['rnks'])
  R['rnks'][kind] = 0
  return R

def randompairingtournament(P):
  n = len(P['P'])
  W = [0] * n
  M = enumeration(P)
  for ind in range( n//2 -1 ):
    m = match(2 * ind, 2 * ind +1, 0.5)
    if m == None:
      m = 2 * ind
    W[m] = 1
  R = randomselectiontournament(M, W)
  return R

## Testing Code ✅

In [ ]:
P = unrankedplayers(11)
W = [1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1]
R = randomselectiontournament(P, W)
print('Random Selection Tournament\n')
print(R)
print('\n')
P = unrankedplayers(11)
print('Random Pairing Tournament\n')
R = randompairingtournament(P)
print(R)

Random Selection Tournament

{'P': ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10'], 'rnks': [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]}


Random Pairing Tournament

{'P': ['P0', 'P1', 'P8', 'P7', 'P4', 'P2', 'P5', 'P6', 'P3', 'P9', 'P10'], 'rnks': [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]}


## Single Elimination Tournament

- Single elimination tournaments are one of the most common formats for competitive tournaments. In this format, each participant plays a single game or match against another participant. The winner of each match moves on to the next round, while the loser is eliminated from the tournament. This process is repeated until only one player remains, who is declared the winner of the tournament.

### Standard Seeding

Standard seeding is a method used to determine the initial match-ups in a tournament based on the rankings of the participating teams or individuals. The seedings are usually determined by a ranking system that takes into account the performance or strength of each participant prior to the tournament. In a standard seeding, the highest-ranked participant is matched up with the lowest-ranked participant, the second-highest ranked participant is matched up with the second-lowest-ranked participant, and so on. This method is used to create a balanced and fair competition, ensuring that the strongest participants are not matched up against each other in the early stages of the tournament.

In [ ]:
def internalstandardseeding(R, np, alpha, omega):
  if alpha == omega:
    return R
  if R['rnks'][alpha] == -1:
    R['rnks'][alpha] = np - 1 - R['rnks'][omega]
  else:
    R['rnks'][omega] = np - 1 - R['rnks'][alpha]
  mu = int((omega - alpha - 1) / 2)
  R = internalstandardseeding(R, 2 * np, alpha, alpha + mu) # Bisect into two and work on each segment separately
  R = internalstandardseeding(R, 2 * np, alpha + mu + 1, omega)
  return R

def standardseeding(n):
  try:
    assert n >= 2
  except:
    raise ValueError('n cannot be less than 2')

  try:
    assert np.log2(n) % 1 == 0
  except:
    raise ValueError('n should be a power of 2!')
  R = unrankedplayers(n)
  R['rnks'] = [-1] * n
  R['rnks'][0] = 0
  return internalstandardseeding(R, 2, 0, n - 1)

## Testing Code ✅

In [ ]:
R = standardseeding(16)
print(R)

{'P': ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15'], 'rnks': [0, 15, 8, 7, 4, 11, 12, 3, 2, 13, 10, 5, 6, 9, 14, 1]}


### Ordered Standard Seeding

In this method, the players or teams are ranked according to their past performance or a predetermined criterion. The highest-ranked player or team is seeded first, followed by the second-highest ranked player or team, and so on until all players or teams are seeded.

In [ ]:
def internalorderedstandardseeding(R, np, alpha, omega):
  if alpha == omega:
    return R
  mu = int((omega - alpha - 1) / 2)
  R = internalorderedstandardseeding(R, 2 * np, alpha, alpha + mu)
  R['rnks'][alpha + mu + 1] = np - 1 - R['rnks'][alpha]
  R = internalorderedstandardseeding(R, 2 * np, alpha + mu + 1, omega)
  return R

def orderedstandardseeding(n):
  try:
    assert n >= 2
  except:
    raise ValueError('n cannot be less than 2')

  try:
    assert np.log2(n) % 1 == 0
  except:
    raise ValueError('n should be a power of 2!')
  R = unrankedplayers(n)
  R['rnks'][0] = 0
  return internalorderedstandardseeding(R, 2, 0, n-1)

In [ ]:
R = orderedstandardseeding(16)
print(R)

{'P': ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15'], 'rnks': [0, 15, 7, 8, 3, 12, 4, 11, 1, 14, 6, 9, 2, 13, 5, 10]}


### Equitable Seeding

Equitable seeding is a concept in tournament design that involves creating fair and balanced brackets by assigning initial seedings to competitors based on their relative strength or ability. The goal is to ensure that the strongest competitors do not face each other too early in the tournament, while also preventing weaker competitors from being immediately eliminated. Equitable seeding can help to increase the overall competitiveness of the tournament, as well as the perceived fairness of the results.



In [ ]:
def bitreverse(x, w):
  xbin = bin(x)[2:]
  wl = len(xbin)
  if wl < w:
    xbin = '0' * (w-wl) + xbin # Concatenate as string
  xrev = xbin[::-1] # Reverse ordering
  # print(xbin, xrev)
  return int('0b' + '0' * (32 - w) + xrev, 2) # Convert to integer

def equitableseeding(n):
  w = 1 + np.floor(np.log(n-1))
  R = unrankedplayers(n)
  for ind in range(n):
    R['rnks'][ind] = bitreverse(ind, int(w))
  return R

## Testing Code ✅

In [ ]:
print(bitreverse(3, 4))
R = equitableseeding(16)
print(R)

12
{'P': ['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15'], 'rnks': [0, 4, 2, 6, 1, 5, 3, 7, 1, 9, 5, 13, 3, 11, 7, 15]}


## Scoring Tournaments

- A Round Robin tournament is a competition in which each participant plays against every other participant in the tournament. This means that if there are n participants, each participant plays n-1 matches. The matches are usually played in a fixed order or scheduled in advance so that there is no confusion.

In [ ]:
def roundrobin(r, n):
  R = unrankedplayers(n)
  R['P'][n-1] = 'P' + str(r)
  np = n
  if n % 2 == 0:
    R['P'][n-2] = 'P' + str(n-1)
    np = n - 1
  for k in range(1, (np-1)//2 + 1):
    ind = 2 * (k - 1)
    R['P'][ind] = 'P' + str((r + k) % np)
    R['P'][ind + 1] = 'P' + str((r + np - k) % np)
  return R

def pairings(R):
  pairs = []
  ln = len(R['P'])
  resting = None
  if ln % 2 == 1:
    resting = R['P'][ln - 1]
    ln -= 1
  for ind in range(ln//2):
    print(ln)
    pairs.append((R['P'][2 * ind], R['P'][2 * ind + 1]))
  return pairs, resting


## Testing Code ✅

In [ ]:
R = roundrobin(2, 7)
print(pairings(R))

6
6
6
([('P3', 'P1'), ('P4', 'P0'), ('P5', 'P6')], 'P2')


# **Programming Assignment #4**

## Fall 2023/24 Semester

1. Your professor, while looking back at how badly he implemented dictionary structure that holds the player data feels frustrated. He thinks it may just be much easier to implement a `Player` class with appropriate variables (`name`, `rank`, `peerWinner`), the getter and setter functions (`getrank`, `setrank` etc.) and perhaps other utilities like `copy` Your first task is to implement this class.

2. You will then implement `unrankedplayers()`, `initialrankadjustment()` and `pyramidmatch()` functions.

3. Finally, you will implement `kingofthehill()` tournament given on p. 112 (Algorithm 5.5) of Jouni and Smed.

**GRADING:**
- `Player` class is implemented (5/100)
- `unrankedplayers()` using `Player` class (5/100)
- `initialrankadjustment()` using `Player` class (5/100)
- `pyramidmatch()` using the `Player` class (5/100)
- `kingofthehill()` function works with default arguments (40/100)
- Code carries out necessary checks on inputs via proper assertions (20/100)
- Code is properly commented, explained and documented (20/100)


**Notes:**

- You are free to use chatGPT for help. However, you should not use code generated by chatGPT directly and you should mention which part of the code is chatGPT in the comments. Your code will be checked for verbatim copying from chatGPT.
- You are not allowed to use external modules/libraries in your implementations except those already in the notebook, but you can use them for testing your code.
- You will upload your submission as a Python notebook to ODTUClass.

(c) Huseyin Hacihabiboglu, 2022-2023